In [ ]:
import pandas_gbq
import pandas as pd

In [ ]:
categories = ['pts','ast','reb','3pm']

outcome_data = {}

for i in categories:

    query = f""" 
        select
            cl.*,cs.Over,cs.Under
            from `capstone_data.{i}_cl_outcome` cl
            inner join `capstone_data.{i}_classifications` cs
                on cs.player = cl.player and date(cs.Date_Updated) = cl.game_date
    """

    data = pandas_gbq.read_gbq(query,project_id='miscellaneous-projects-444203')

    outcome_data[i] = data

In [20]:
roi_by_category = {}

for i in ['pts', 'ast', 'reb', '3pm']:
    df = outcome_data[i].copy()  # grab the DataFrame for this category
    
    # Deduplicate: keep one row per player-game
    df = df.sort_values('proba', ascending=False)
    df = df.drop_duplicates(subset=['player', 'game_date'])
    
    # Rename odds columns just in case
    df = df.rename(columns={'Over': 'over_odds', 'Under': 'under_odds'})
    
    # Profit calculation
    def compute_profit(row):
        odds = row['over_odds'] if row['recommendation'] == 'Over' else row['under_odds']
        if row['recommendation'] == row['result']:
            return 100 if odds < 0 else odds
        else:
            return -abs(odds) if odds < 0 else -100

    df['profit'] = df.apply(compute_profit, axis=1)

    # Risk calculation (for ROI)
    def compute_risk(row):
        odds = row['over_odds'] if row['recommendation'] == 'Over' else row['under_odds']
        return 100 if odds < 0 else abs(odds)

    df['risk'] = df.apply(compute_risk, axis=1)

    # ROI tracking
    roi_tracker = (
        df.groupby('game_date')
        .agg(
            bets_placed=('player', 'count'),
            total_profit=('profit', 'sum'),
            total_risk=('risk', 'sum'),
            roi_percent=('profit', lambda x: round(x.sum() / df.loc[x.index, 'risk'].sum() * 100, 2))
        )
        .reset_index()
        .sort_values('game_date')
    )

    roi_by_category[i] = roi_tracker  # store result



In [21]:
for i in categories:
    print(i)
    print(roi_by_category[i])

pts
    game_date  bets_placed  total_profit  total_risk  roi_percent
0  2025-04-03           45           130        4500         2.89
1  2025-04-04           67          -665        6700        -9.93
ast
    game_date  bets_placed  total_profit  total_risk  roi_percent
0  2025-04-03           21           -46        2178        -2.11
1  2025-04-04           25          -441        2682       -16.44
reb
    game_date  bets_placed  total_profit  total_risk  roi_percent
0  2025-04-03           41           240        4238         5.66
1  2025-04-04           47           116        4852         2.39
3pm
    game_date  bets_placed  total_profit  total_risk  roi_percent
0  2025-04-03           20          -471        2103       -22.40
1  2025-04-04           13           199        1389        14.33


In [ ]:
for cat, df in roi_by_category.items():
    total_roi = round(df['total_profit'].sum() / df['total_risk'].sum() * 100, 2)
    print(f"{cat.upper()} total ROI: {total_roi}%")
